<a target="_blank" href="https://colab.research.google.com/github/michalis0/Cloud-and-Advanced-Analytics/blob/main/week_12_GPT/fine-tunning.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

<h1 align="center"> LAB - GPT</h1>

<div>
<td> 
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/2/2b/Logo_Universit%C3%A9_de_Lausanne.svg/2000px-Logo_Universit%C3%A9_de_Lausanne.svg.png" style="padding-right:10px;width:240px;float:left"/></td>
<h2 style="white-space: nowrap">Cloud and Advanced Analytics </h2></td>
<hr style="clear:both">
<p style="font-size:0.85em; margin:2px; text-align:justify">

</div>

### LINKS AND RESSOURCES
- https://platform.openai.com/docs/guides/fine-tuning
- https://norahsakal.com
- https://spotintelligence.com/2023/04/21/fine-tuning-gpt-3/

## INTRODUCTION

A language model is a type of artificial intelligence algorithm that can generate and understand human language. It works by predicting the next word or sequence of words in a given piece of text, based on the words that have come before it.

GPT-3 (Generative Pre-trained Transformer 3) is a large, powerful language model developed by OpenAI that has been trained on a massive corpus of text data. It has been trained using a transformer architecture, which is a type of neural network that is designed to handle sequential data, such as natural language.

Because of its massive size and extensive training, GPT-3 is capable of performing a wide variety of language-based tasks, including text generation, text completion, translation, and more.

However, GPT-3 is a general-purpose language model, which means it has been trained on a broad range of data and doesn’t have specific knowledge about any particular domain or task. This is where fine-tuning comes in.

By fine-tuning a GPT-3 model on a specific task or domain, you can customize it to perform better on that task, making it more accurate and effective. This is done by feeding the model with examples specific to the task, which allows it to learn the patterns and rules that are relevant to that task.

### What is fine tuning?
Fine-tuning is a process of retraining a pre-trained model on a specific task or domain. The idea behind fine-tuning is to leverage the pre-existing knowledge of a pre-trained model and apply it to a more specific task. Fine-tuning allows you to customize a pre-trained model to your particular needs, which can improve its performance on the given task.

### Outline
1. Get OpenAI API key

2. Create training data

3. Check the training data

4. Upload training data

5. Fine-tune model

6. Check fine-tune progress

7. Save fine-tuned model

8. Test the new model on a new prompt

In [ ]:
import json
import openai

## 1. OpenAI Key

In [ ]:
api_key ="YOUR_OPENAI_API_KEY"
openai.api_key = api_key

## 2. Create Training Data

Make sure to end each prompt with a suffix szch as `->` and to end each completion with a suffix as well such as `.\n`

In [ ]:
data_file = [{
    "prompt": "Prompt ->",
    "completion": " Ideal answer.\n"
},{
    "prompt":"Prompt ->",
    "completion": " Ideal answer.\n"
}]

## 3. Save dict as JSONL file

In [ ]:
file_name = "training_data.jsonl"

with open(file_name, 'w') as outfile:
    for entry in data_file:
        json.dump(entry, outfile)
        outfile.write('\n')

In [ ]:
!openai tools fine_tunes.prepare_data -f training_data.jsonl

## 4. Upload file to your OpenAI account

In [ ]:
upload_response = openai.File.create(
  file=open(file_name, "rb"),
  purpose='fine-tune'
)
upload_response

### Save file name

In [ ]:
file_id = upload_response.id
file_id

## 5. Fine-tune a model

The default model is `Curie` which is the most powerful one. You can also use `davinci` or `babbage` which are less powerful but cheaper.

In [ ]:
model="davinci"
fine_tune_response = openai.FineTune.create(training_file=file_id, model=model)
fine_tune_response

## 6. Check the status of the fine-tuning process

The fine_tune_response refers to the response object obtained after initiating the fine-tuning process. This code snippet allows you to monitor and track the progress and status of the fine-tuning job by examining the events associated with it. The fine_tune_events variable will contain the list of events retrieved.

In [ ]:
fine_tune_events = openai.FineTune.list_events(id=fine_tune_response.id)
fine_tune_events

By executing this code, you can obtain the current status, configuration, and other relevant details of the fine-tuning process. The retrieve_response variable will contain the response object with the retrieved information.

In [ ]:
retrieve_response = openai.FineTune.retrieve(id=fine_tune_response.id)
retrieve_response

## 7. Save the fine-tuned model

#### Troubleshooting fine_tuned_model as null

During the fine-tuning process, the __fine_tuned_model__ key may not be immediately available in the fine_tune_response object returned by `openai.FineTune.create().`

To check the status of your fine-tuning process, you can call the `openai.FineTune.retrieve()` function and pass in the __fine_tune_response.id__. This function will return a JSON object with information about the training status, such as the current epoch, the current batch, the training loss, and the validation loss.

After the fine-tuning process is complete, you can check the status of all your fine-tuned models by calling openai.FineTune.list(). This will list all of your fine-tunes and their current status.

Once the fine-tuning process is complete, you can retrieve the fine_tuned_model key by calling the openai.FineTune.retrieve() function again and passing in the fine_tune_response.id. This will return a JSON object with the key fine_tuned_model and the ID of the fine-tuned model that you can use for further completions.

### Option 1

In [ ]:
# If fine_tune_response.fine_tuned_model != None then the key fine_tuned_model is availble from the fine_tune_response object
if fine_tune_response.fine_tuned_model != None:
    fine_tuned_model = fine_tune_response.fine_tuned_model

### Option 2

In [ ]:
# If fine_tune_response.fine_tuned_model == None: you can get the fine_tuned_model by listing all fine-tune events
if fine_tune_response.fine_tuned_model == None:
    fine_tune_list = openai.FineTune.list()
    fine_tuned_model = fine_tune_list['data'][0].fine_tuned_model

### Option 3

In [ ]:
# If fine_tune_response.fine_tuned_model == None: you can get the fine_tuned_model key by retrieving the fine-tune job
if fine_tune_response.fine_tuned_model == None:
    fine_tuned_model = openai.FineTune.retrieve(id=fine_tune_response.id).fine_tuned_model

## 8. Test the fine-tuned model

In [ ]:
new_prompt = "NEW PROMPT ->"

In [ ]:
answer = openai.Completion.create(
  model=fine_tuned_model,
  prompt=new_prompt,
  max_tokens=10, # Change amount of tokens for longer completion
  temperature=0
)
answer['choices'][0]['text']